# 1. Filter the GLDv2full embeddings by the landmark_ids of GLDv2clean

In [3]:
import numpy as np
import pandas as pd

In [4]:
train_kaggle=pd.read_csv('./input/train_kaggle.csv')

In [5]:
train=pd.read_csv('./input/train.csv')

In [7]:
embeddings= np.load("./input/config1_train_embeddings.npy")

In [8]:
embeddings.shape

(4132914, 512)

In [7]:
from tqdm.notebook import tqdm

In [13]:
landmark_ids=train_kaggle.landmark_id.unique()

In [17]:
train['landmark_id'].isin(landmark_ids).value_counts()

True     3223078
False     909836
Name: landmark_id, dtype: int64

In [24]:
train_filtered=train[train['landmark_id'].isin(landmark_ids)]

In [23]:
embeddings_filtered=embeddings[train['landmark_id'].isin(landmark_ids)]

In [30]:
train_full_filtered=train_filtered.reset_index(drop=True)

In [33]:
train_full_filtered.to_csv('./input/train_full_filtered.csv')

In [34]:
np.save(f"./input/config1_train_embeddings_filtered", embeddings_filtered)

## 1.1 Split the filtered embeddings (3,223,078) into 100 dictionaries for memory saving in the inference process

In [9]:
train_full_filtered=pd.read_csv('./input/train_full_filtered.csv')

In [10]:
targets=train_full_filtered.landmark_id.values

In [11]:
idx=np.array(train_full_filtered.index.tolist())

In [12]:
config1_train_embeddings_filtered= np.load("./input/config1_train_embeddings_filtered.npy")

In [13]:
config1_train_embeddings_filtered

array([[-0.385878  , -0.45518425, -0.12521571, ..., -0.17065661,
         0.885234  , -0.58148366],
       [-0.54030168, -0.03456247, -0.0021289 , ...,  0.23576169,
        -0.10906727, -0.03044808],
       [ 0.02245048,  0.38831481, -0.41250756, ..., -0.36589488,
        -0.19105846, -0.4766688 ],
       ...,
       [ 1.01260626,  0.0391771 , -0.33502147, ..., -0.22563982,
        -0.29557696, -0.22145835],
       [ 0.35365015, -0.16269273, -0.51898676, ..., -0.04626337,
        -0.4416995 , -0.52597457],
       [-0.32673448,  0.19875242, -0.25290754, ...,  0.0046272 ,
         0.83779567,  0.21488364]])

In [15]:
config1_train_embeddings_filtered.shape[0]

3223078

In [16]:
from tqdm.notebook import tqdm
import pickle

In [17]:
total_num=config1_train_embeddings_filtered.shape[0]
split_num=100
unit_num=total_num//split_num

for i in tqdm(range(split_num)):
    dic={}
    start=unit_num*i
    
    if i==99:
        end=total_num
    else:
        end=unit_num*(i+1)
        
    dic={'idx':idx[start:end],
        'embeddings':config1_train_embeddings_filtered[start:end],
        'targets':targets[start:end]}
    
    with open(f'./out_split100/out_tr_filter_{i}.p', 'wb') as handle:
        pickle.dump(dic, handle)

  0%|          | 0/100 [00:00<?, ?it/s]

In [18]:
dic

{'idx': array([3190770, 3190771, 3190772, ..., 3223075, 3223076, 3223077]),
 'embeddings': array([[ 0.265504  ,  0.55583251, -0.36087981, ...,  0.12538739,
          0.04557478,  0.27489191],
        [-0.41380179,  0.08416938,  0.25622481, ..., -0.02443341,
         -0.78822869, -0.25966278],
        [ 0.07103499, -0.2964066 ,  0.22915561, ..., -0.32925749,
         -0.19890943,  0.31845906],
        ...,
        [ 1.01260626,  0.0391771 , -0.33502147, ..., -0.22563982,
         -0.29557696, -0.22145835],
        [ 0.35365015, -0.16269273, -0.51898676, ..., -0.04626337,
         -0.4416995 , -0.52597457],
        [-0.32673448,  0.19875242, -0.25290754, ...,  0.0046272 ,
          0.83779567,  0.21488364]]),
 'targets': array([150984,  59880, 192594, ..., 191243, 145760,  34698])}

In [14]:
# !gsutil -m cp ./out_split100/* gs://kaggle-tropicbird/embeddings/filtered/dic_split100

Copying file://./out_split10/out_tr_filter_0.p [Content-Type=text/x-pascal]...
Copying file://./out_split10/out_tr_filter_1.p [Content-Type=text/x-pascal]...  
Copying file://./out_split10/out_tr_filter_2.p [Content-Type=text/x-pascal]...  
Copying file://./out_split10/out_tr_filter_3.p [Content-Type=text/x-pascal]...  
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite ob

# 2. Split the full embeddings (4,132,914) into 100 dictionaries for memory saving in the inference process

In [ ]:
train=pd.read_csv('./input/train.csv')

In [3]:
train

,id,url,landmark_id
0,6e158a47eb2ca3f6,https://upload.wikimedia.org/wikipedia/commons...,142820
1,202cd79556f30760,http://upload.wikimedia.org/wikipedia/commons/...,104169
2,3ad87684c99c06e1,http://upload.wikimedia.org/wikipedia/commons/...,37914
3,e7f70e9c61e66af3,https://upload.wikimedia.org/wikipedia/commons...,102140
4,4072182eddd0100e,https://upload.wikimedia.org/wikipedia/commons...,2474
...,...,...,...
4132909,fc0f007893b11ba7,https://upload.wikimedia.org/wikipedia/commons...,172138
4132910,39aad18585867916,https://upload.wikimedia.org/wikipedia/commons...,162860
4132911,fd0725460e4ebbec,https://upload.wikimedia.org/wikipedia/commons...,191243
4132912,73691ae29e24ba19,https://upload.wikimedia.org/wikipedia/commons...,145760


In [4]:
targets=train.landmark_id.values

In [5]:
targets.shape

(4132914,)

In [6]:
idx=np.array(train.index.tolist())

In [7]:
idx

array([      0,       1,       2, ..., 4132911, 4132912, 4132913])

In [8]:
# embeddings= np.load("./input/config1_train_embeddings.npy")
embeddings= np.load("./input/config7_train_embeddings.npy")

In [9]:
embeddings.shape

(4132914, 512)

In [11]:
4132914//100

41329

In [13]:
from tqdm.notebook import tqdm
import pickle
for i in tqdm(range(100)):
    dic={}
    
    start=41329*i
    
    if i==99:
        end=4132914
    else:
        end=41329*(i+1)
        
    dic={'idx':idx[start:end],
        'embeddings':embeddings[start:end],
        'targets':targets[start:end]}
    
    
#     with open(f'./out_split100_full/out_tr_filter_{i}.p', 'wb') as handle:
    with open(f'./out_split100_full_config7/out_tr_filter_{i}.p', 'wb') as handle:
        pickle.dump(dic, handle)

  0%|          | 0/100 [00:00<?, ?it/s]

In [14]:
!gsutil -m cp ./out_split100_full_config7/* gs://kaggle-tropicbird/embeddings/config7_split100_full

Copying file://./out_split100_full_config7/out_tr_filter_1.p [Content-Type=text/x-pascal]...
Copying file://./out_split100_full_config7/out_tr_filter_0.p [Content-Type=text/x-pascal]...
Copying file://./out_split100_full_config7/out_tr_filter_10.p [Content-Type=text/x-pascal]...
Copying file://./out_split100_full_config7/out_tr_filter_11.p [Content-Type=text/x-pascal]...
Copying file://./out_split100_full_config7/out_tr_filter_12.p [Content-Type=text/x-pascal]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmo